# ノード操作のスケジュールを変更する
---

workerノードのスケジュール設定を変更します。

## ノード操作を行う環境の指定

VCノード操作コンテナを実行する環境のansibleグループを指定します。

既存のグループ名を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上記のセルに表示されたグループ名から電源操作コンテナを実行しているグループ名を次のセルで指定してください。

In [ ]:
# (例)
# group = 'CoursewareHub'
# group = 'powerctl'
# group = 'scaleout'

group = 

### チェック

指定されたグループ名が妥当なものであることをチェックします。

設定されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(group)
target_group = group if "rsc_yml" not in gvars else f"{group}_manager"

指定されたノードにansibleでアクセスできることを確認します。

In [ ]:
!ansible {target_group} -m ping

スケジュールの設定ファイルが対象ノードに存在していることを確認します。

In [ ]:
!ansible {target_group} -a 'test -f {gvars["vcnode_ctl_dir"]}/schedule.json'

## パラメータの指定

VCノードのスケジュールを設定するのに必要となるパラメータを指定します。

### VCCアクセストークンの入力

VCノードを操作するためにVC Controller(VCC)のアクセストークンが必要となります。
次のセルを実行すると表示される入力枠にVCCのアクセストークンを入力してください。

> アクセストークン入力後に Enter キーを押すことで入力が完了します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

入力されたアクセストークンが正しいことを、実際にVCCにアクセスして確認します。

In [ ]:
from common import logsetting
from vcpsdk.vcpsdk import VcpSDK

vcp = VcpSDK(vcc_access_token)

上のセルの実行結果がエラーとなり以下のようなメッセージが表示されている場合は、入力されたアクセストークンに誤りがあります。

```
config vc failed: http_status(403)
2023/XX/XX XX:XX:XX UTC: VCPAuthException: xxxxxxx:token lookup is failed: permission denied
```

エラーになった場合はこの節のセルを全て `unfreeze` してから、もう一度アクセストークンの入力を行ってください。

### workerノードの状態変更のスケジュールを指定する

それぞれの利用状況ごとに、どのようなノード状態を設定するかを指定します。

In [ ]:
!ansible {target_group} -a 'cat {gvars["vcnode_ctl_dir"]}/schedule.json'

#### 授業日

授業の曜日を指定してください。

In [ ]:
# (例)
# schedule_day_of_week = '月'
# schedule_day_of_week = 'Tue'
# schedule_day_of_week = 'Wednesday'


schedule_day_of_week = 

#### 授業時間中のノード状態

利用状況を「授業時間中」にする開始時間を指定してください。通常は実際の授業開始時刻の３０〜６０分前の時刻を指定します。

In [ ]:
# (例)
# schedule_time_0 = '8:30'

schedule_time_0 = 

「授業時間中」における各ノード状態に設定するノード数を指定します。通常は「利用可能」(active)のノード数にworkerノード数を指定し他の状態には0を指定します。



電源操作を行うスケジュールの場合は、ここで指定するworkerノード数の合計が現在のworkerノード数と一致している必要があります。またノードすうを変更するスケジュールの場合は、ここで指定するworkerノード数の合計が`all_worker_ipaddresses`に指定したIPアドレス数と一致している必要があります。ノード数の合計値の条件を確認します。次のセルの出力結果に、指定するノード数の合計値が一致する必要があります。

In [ ]:
if "all_worker_ipaddresses" in gvars:
    worker_count = len(gvars["all_worker_ipaddresses"])
else:
    ug = vcp.get_ugroup(gvars["ugroup_name"])
    ut = ug.get_unit("worker")
    worker_count = len(ut.find_nodes())
print(worker_count)

ノード状態ごとのノード数を指定してください。各状態のノード数の合計が上のセルの出力結果と一致するようにしてください。

In [ ]:
# (例)
# node_availability_0 = {
#     "active": 3,    # 利用可能状態のノード数
#     "pause": 0,     # 新規利用不可状態のノード数
#     "drain": 0,       # 利用不可状態のノード数
# }

node_availability_0 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 授業時間後のノード状態

利用状況を「授業時間後」にする開始時間を指定してください。通常は実際の授業終了時刻の３０〜６０分後の時刻を指定します。

In [ ]:
# (例)
# schedule_time_1 = '11:00'

schedule_time_1 = 

「授業時間後」における各ノード状態に設定するノード数を指定します。通常は、少数の「利用可能」(active)のノード以外は「新規利用不可」(pause)となるようにノード数を指定します。各状態のノード数の合計がworkerノードに一致するようにしてください。

In [ ]:
# (例)
# node_availability_1 = {
#     "active": 1,    # 利用可能状態のノード数
#     "pause": 2,     # 新規利用不可状態のノード数
#     "drain": 0,       # 利用不可状態のノード数
# }

node_availability_1 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 授業時間外のノード状態

利用状況を「授業時間外」にする開始時間を指定してください。通常は実際の授業終了時刻の数時間後の時刻を指定します。

In [ ]:
# (例)
# schedule_time_2 = '13:00'

schedule_time_2 = 

「授業時間外」における各ノード状態に設定するノード数を指定します。通常は、少数の「利用可能」(active)のノード以外は「利用不可」(drain)となるようにノード数を指定します。各状態のノード数の合計がworkerノードに一致するようにしてください。

In [ ]:
# (例)
# node_availability_2 = {
#     "active": 1,    # 利用可能状態のノード数
#     "pause": 0,     # 新規利用不可状態のノード数
#     "drain": 2,       # 利用不可状態のノード数
# }

node_availability_2 = {
    "active": ,
    "pause": ,
    "drain": ,
}

#### 指定されたパラメータの保存

指定されたパラメータをgroup_varsに保存します。

パラメータの保存を行う前に簡単なチェックを行います。

In [ ]:
import re

if schedule_day_of_week.lower() not in [
    "sun", "mon", "tue", "wed", "thu", "fri", "sat",
    "sunday", "monday", "tuesday", "wednesday", "thursday", "friday", "saturday",
    "日", "月", "火", "水", "木", "金", "土",
]:
    raise RuntimeError(f"曜日指定が正しくない: {schedule_day_of_week}")

for x in [schedule_time_0, schedule_time_1, schedule_time_2]:
    if (re.fullmatch(r"(?:[01]?\d|2[0-3]):[0-5]?\d", x, re.A) is None):
        raise RuntimeError(f"時刻指定が正しくない: {x}")

for x in [node_availability_0, node_availability_1, node_availability_2]:
    if sum(x.values()) != worker_count:
        raise RuntimeError(f"ノード数の合計がworkerノード数と一致していない: {x}")

group_varsにパラメータを保存します。

In [ ]:
%run scripts/group.py
schedule = [
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_0.split(":", 2)[0]),
            "minute": int(schedule_time_0.split(":", 2)[1]),
        },
        "availability": node_availability_0,
    },
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_1.split(":", 2)[0]),
            "minute": int(schedule_time_1.split(":", 2)[1]),
        },
        "availability": node_availability_1,
    },
    {
        "schedule": {
            "day_of_week": schedule_day_of_week.lower(),
            "hour": int(schedule_time_2.split(":", 2)[0]),
            "minute": int(schedule_time_2.split(":", 2)[1]),
        },
        "availability": node_availability_2,
    },
]
update_group_vars(
    target_group if not target_group.startswith(gvars["ugroup_name"]) else gvars["ugroup_name"],
    vcnode_schedule=schedule
)

## 設定ファイルの配置

workerノード状態の設定スケジュールを記した設定ファイル`schedule.json`を配置します。

In [ ]:
!ansible {target_group} -D -m template -a \
    'src=playbooks/roles/worker-schedule/templates/schedule.json.j2 \
    dest={{{{vcnode_ctl_dir}}}}/schedule.json backup=true'

systemdタイマーの具体的な次の実行タイミングを確認するためにsystemdタイマーの状況を確認します。`NEXT`, `LEFT`の欄を確認してください。

In [ ]:
flags = '--user' if not target_group.startswith(gvars["ugroup_name"]) else ''
!ansible {target_group} -a 'systemctl {flags} list-timers cwh-worker-node.timer'

サービスのログ出力を確認します。

In [ ]:
become = "-b" if "master_fqdn" in gvars else ""
!ansible {target_group} {become} -a 'journalctl {flags} -u cwh-worker-node -n 30'

workerノードの現在の状態を確認します。

In [ ]:
ug = vcp.get_ugroup(gvars["ugroup_name"])
ut = ug.get_unit("worker")
ut.df_nodes()